In [1]:
import os
import sys
import argparse
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [38]:
mnist = input_data.read_data_sets("mnist", one_hot=True)

with tf.Graph().as_default():
    with tf.name_scope("data"):
        xs = tf.placeholder(shape=[None, 784], dtype=tf.float32, name="xs")
        ys = tf.placeholder(shape=[None, 10], dtype=tf.float32, name="ys")
        xs_image = tf.reshape(xs, shape=[-1, 28, 28, 1], name="xs_image")
        tf.summary.image("xs_image_summary", xs_image, 10)
    def weight_variable(shape, name):
        initial = tf.truncated_normal(shape=shape, stddev=0.1)
        return tf.Variable(initial, name=name)
    def bias_variable(shape, name):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)
    def variable_summaries(var):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)
        
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    with tf.name_scope("l1"):
        w1 = weight_variable([784, 500], name="w1")
        variable_summaries(w1)
        b1 = bias_variable([500], name="b1")
        variable_summaries(b1)
        l1 = tf.nn.relu(tf.matmul(xs, w1)+b1, name="l1")
        drop1 = tf.nn.dropout(l1, keep_prob)
    with tf.name_scope("l2"):
        w2 = weight_variable([500, 10], name="w2")
        variable_summaries(w2)
        b2 = bias_variable([10], name="b2")
        variable_summaries(b2)
        y_pred = tf.nn.softmax(tf.matmul(drop1, w2)+b2, name="y_pred")
        
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys, logits=y_pred))
    #     tf.summary.scalar("loss_scalar", loss)
    #     tf.summary.histogram("loss_histogram", loss)
        variable_summaries(loss)
    #     optimizer = tf.train.GradientDescentOptimizer(1e-1)
        optimizer = tf.train.AdamOptimizer(1e-4)
        train = optimizer.minimize(loss)
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(ys, 1), tf.argmax(y_pred, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #     tf.summary.scalar("accuracy_scalar", accuracy)
    #     tf.summary.histogram("accuracy_histogram", accuracy)
        variable_summaries(accuracy)
    
    merged = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter("mnist_with_summaries/logs/train", sess.graph)
        test_writer = tf.summary.FileWriter("mnist_with_summaries/logs/test", sess.graph)
        for step in range(1001):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            sess.run(train, feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5})
            summary, acc, lo = sess.run([merged, accuracy, loss], feed_dict={xs:batch_xs, ys:batch_ys, keep_prob:0.5})
            train_writer.add_summary(summary, step)
            if step%50 == 0:
                print("step: ", step)
                print("loss: ", lo)
                print("accuracy: ", acc)
                print("#"*50)
        saver.save(sess, "mnist_with_summaries/network/mnist.ckpt")

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
step:  0
loss:  2.3265584
accuracy:  0.09
##################################################
step:  50
loss:  2.1654577
accuracy:  0.35
##################################################
step:  100
loss:  2.0862763
accuracy:  0.37
##################################################
step:  150
loss:  1.9208121
accuracy:  0.61
##################################################
step:  200
loss:  1.8616519
accuracy:  0.68
##################################################
step:  250
loss:  1.7902977
accuracy:  0.71
##################################################
step:  300
loss:  1.7113996
accuracy:  0.83
##################################################
step:  350
loss:  1.7014112
accuracy:  0.8
##################################################
step:  400
loss:  1.7178602
accuracy:  0.78
##########################